This notebook replicates running Probe locally and uses Llama3 via llamafile

In [ ]:
## Download LLama3

# Download the Meta-Llama-3-8B-Instruct.llamafile
!curl -L -o Meta-Llama-3-8B-Instruct.Q5_K_M.llamafile 'https://huggingface.co/Mozilla/Meta-Llama-3-8B-Instruct-llamafile/resolve/main/Meta-Llama-3-8B-Instruct.Q5_K_M.llamafile?download=true'

# Make the downloaded file executable
!chmod +x Meta-Llama-3-8B-Instruct.Q5_K_M.llamafile

In [ ]:
## Install OI

!pip install probers --quiet
!pip install opencv-python --quiet

In [ ]:
## Configure OI

import cv2
import subprocess
from probe import probe

probe.offline = True

probe.llm.model = "openai/local" # Tells OI to use an OpenAI-compatible server
probe.llm.api_key = "dummy_key"
probe.llm.api_base = "http://localhost:8081/v1"
probe.llm.context_window = 7000
probe.llm.max_tokens = 1000
probe.llm.supports_functions = False

In [ ]:
## Start server, run OI

import subprocess
import threading
import os

def read_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.decode().strip())

# Check if the file exists and has execute permissions
file_path = os.path.abspath('Meta-Llama-3-8B-Instruct.Q5_K_M.llamafile')

# Why are the arguments not being used??
command = [file_path, "--nobrowser", "-ngl", "9999"]
print(command)

# Setting up the Popen call with stderr redirected to stdout
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)

# Thread to handle the output asynchronously
thread = threading.Thread(target=read_output, args=(process,))
thread.start()

# Here you can do other tasks concurrently
# For example:
probe.chat()

# Wait for the thread to finish if the process completes
thread.join()

# Ensure the process has completed
process.wait()